In [ ]:
#setup
bucket = 'baby-data'

import boto3
import re
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

import numpy as np
import pandas as pd
import sagemaker
from sagemaker.predictor import csv_serializer 

#from sklearn import metrics
#from sklearn.metrics import roc_auc_score
#import matplotlib.pyplot as plt   

s3 = boto3.client('s3') 
obj = s3.get_object(Bucket = bucket, Key = 'mergedData.csv') 

data = pd.read_csv(obj['Body'])

numberOfBabies = data['id'].nunique()

#pred_data = pd.DataFrame(columns=['actual', 'predicted'])

In [ ]:
babyNum = 1
innerBabyNum = 1
while babyNum <= numberOfBabies:
    #hold out one baby's data for testing
    test_data = data.loc[data['id'] == babyNum]
    test_data = test_data.drop(['id'], axis=1)
    test_data = test_data.reset_index(drop=True)
    
    #separate the rest of the data into training and validation
    train_data = pd.DataFrame(columns=data.columns)
    validation_data = pd.DataFrame(columns=data.columns)
    
    while innerBabyNum <= numberOfBabies:
        if innerBabyNum == babyNum:
            innerBabyNum = innerBabyNum + 1
        else:
            innerBabyData = data.loc[data['id'] == innerBabyNum]
            innerBabyData = innerBabyData.reset_index(drop=True)
            train, validation = np.split(innerBabyData, [int(0.7*len(innerBabyData))])    
            train_data = train_data.append(train)
            validation_data = validation_data.append(validation)        
            innerBabyNum = innerBabyNum + 1
    validation_data = validation_data.drop(['id'], axis=1)
    train_data = train_data.drop(['id'], axis=1)
    
    train_data.to_csv('train.csv', index=False, header=False)
    validation_data.to_csv('validation.csv', index=False, header=False)
    test_data.to_csv('test.csv', index=False, header=False)

    #copy the file to S3
    boto3.Session().resource('s3').Bucket(bucket).Object('train/train.csv').upload_file('train.csv')
    boto3.Session().resource('s3').Bucket(bucket).Object('validation/validation.csv').upload_file('validation.csv')
    boto3.Session().resource('s3').Bucket(bucket).Object('test/test.csv').upload_file('test.csv')

    container = get_image_uri(boto3.Session().region_name, 'xgboost')

    s3_input_train = sagemaker.s3_input(s3_data = 's3://{}/train'.format(bucket), content_type = 'csv')
    s3_input_validation = sagemaker.s3_input(s3_data = 's3://{}/validation'.format(bucket), content_type = 'csv')
    s3_input_test = sagemaker.s3_input(s3_data = 's3://{}/test'.format(bucket), content_type = 'csv')

    #train
    sess = sagemaker.Session()

    xgb = sagemaker.estimator.Estimator(container,
                                       role,
                                       train_instance_count=1,
                                       train_instance_type='ml.m4.xlarge',
                                       output_path='s3://{}/output'.format(bucket),
                                       sagemaker_session=sess)
    xgb.set_hyperparameters(objective='binary:logistic', 
                            eval_metric='error',
                            alpha=1.5,
                            eta=.05,
                            max_depth=8,
                            min_child_weight=3.7,
                            num_round=100
                           )

    xgb.fit({'train': s3_input_train, 'validation': s3_input_validation, 'test': s3_input_test})
    
    print(baby)
    babyNum = babyNum + 1